# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ITMO][ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
#from catboost import CatBoostClassifier

In [4]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [9]:
class DecisionTreeLeaf:
    """

    Attributes
    ----------
    y : Тип метки (напр., int или str)
        Метка класса, который встречается чаще всего среди элементов листа дерева
    """
    def __init__(self, y):
      classes, counts = np.unique(y, return_counts=True)
      self.y = classes[np.argmax(counts)]
      self.proba = dict(zip(classes, counts / np.sum(counts)))#...

    def predict(self, X):
      return np.full(len(X), self.proba)
        

class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value. 
    """
    def __init__(self, split_dim: int,
                 left, right):
        self.split_dim = split_dim
        self.left = left
        self.right = right
        
    def predict(self, X):
        y = len(X) * [None]
        left_mask = X[:, self.split_dim] == 0
        right_mask = X[:, self.split_dim] == 1
        left_pred = self.left.predict(X[left_mask])
        right_pred = self.right.predict(X[right_mask])
        left  = 0
        right = 0
        for i in range(len(y)):
            if left_mask[i]:
                y[i] = left_pred[left]
                left += 1
            else:
                y[i] = right_pred[right]
                right += 1
        return y

class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        if criterion == "gini":
          self.criterion = gini
        else:
          self.criterion = entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        if max_features == "auto":
          self.max_features = int(np.sqrt(X.shape[1]))
        else: 
          self.max_features = max_features
        self.root = self._build_tree(X, y)

    def _find_best_split(self, X, y): 
        best_gain = -1
        best_split_by = None
        dims = np.random.choice(X.shape[1], self.max_features)
        for dim in dims:
          left_y = y[X[:, dim] == 0]
          if (len(left_y) < self.min_samples_leaf) or (len(y) - len(left_y) < self.min_samples_leaf):
                    continue
          right_y = y[X[:, dim] == 1]
          score = gain(left_y, right_y, self.criterion)
          if score > best_gain:
            best_gain = score
            best_split_by = dim
        return best_split_by      
      
    def _build_tree(self,X, y, depth = 0):
            if self.max_depth is not None and depth >= self.max_depth:
            #if depth >= self.max_depth and self.max_depth is not None or X.shape[0] <= self.min_samples:
              return DecisionTreeLeaf(y)
            best_split_by = self._find_best_split(X, y)
            if best_split_by is None:
                return DecisionTreeLeaf(y)               
            left_mask = X[:, best_split_by] == 0
            right_mask = X[:, best_split_by] == 1
            return DecisionTreeNode(best_split_by,
                                    self._build_tree(X[left_mask], y[left_mask], depth+1),
                                    self._build_tree(X[right_mask], y[right_mask], depth+1))      
     
    
    def predict_proba(self, X: np.ndarray):
        return self.root.predict(X)
        
    def predict(self, X):
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba] 

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [42]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_samples_leaf = min_samples_leaf
        self.n_estimators = n_estimators
        self.trees = []

    def fit(self, X, y):
        self.X = X
        self.y = y
        size = int(X.shape[0] / self.n_estimators)
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        self.indices = indices
        for i in range(self.n_estimators):
            self.trees.append(DecisionTree(X[indices[size*i:size*(i+1)]], y[indices[size*i:size*(i+1)]], self.criterion, self.max_depth, self.min_samples_leaf, self.max_features))
    
    def predict(self, X):
        pred = []
        res = []
        for tree in self.trees:
            pred.append(tree.predict(X))
        pred = np.array(pred)
        for i in range(X.shape[0]):
            classes, counts = np.unique(pred[:, i], return_counts=True)
            res.append(classes[np.argmax(counts)])
        return np.array(res)

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [47]:
def feature_importance(rfc):
  X = rfc.X
  y = rfc.y 
  indices = rfc.indices
  importance = [None]*X.shape[1]
  size = int(X.shape[0] / rfc.n_estimators)  
  for i, tree in enumerate(rfc.trees):
    oob_ind = np.concatenate((indices[:i*size], indices[(i+1)*size:]))
    err_oob = 1 - np.mean(tree.predict(X[oob_ind]) == y[oob_ind])
    for f in range(X.shape[1]):
      copy = np.copy(X[oob_ind])
      np.random.shuffle(copy[ : , f])
      err_oob_j = 1 - np.mean(tree.predict(copy) == y[oob_ind])
      err = err_oob_j - err_oob
      if importance[f] is None:
        importance[f] = err
      else:
        importance[f] = np.mean([importance[f], err])
  return np.array(importance)

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [48]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [-0.00179771 -0.00036658  0.04129203  0.00226927  0.24442893  0.0038284 ]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [30]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [31]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [ ]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

#### Пол

In [ ]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [52]:
#!pip install catboost
from catboost import Pool, CatBoostClassifier
X, y = synthetic_dataset(1000)
model = CatBoostClassifier(iterations=15, learning_rate=0.01, depth=10, loss_function='MultiClass')
model.fit(X, y)
y_pred = model.predict(X).reshape(-1,)


print("Accuracy:", np.mean(y_pred == y))
print("Importance: \n", model.get_feature_importance(prettified=True))

0:	learn: 1.0800326	total: 771us	remaining: 10.8ms
1:	learn: 1.0613431	total: 2.22ms	remaining: 14.4ms
2:	learn: 1.0437625	total: 3.15ms	remaining: 12.6ms
3:	learn: 1.0266410	total: 3.89ms	remaining: 10.7ms
4:	learn: 1.0094088	total: 4.53ms	remaining: 9.06ms
5:	learn: 0.9926312	total: 5.19ms	remaining: 7.78ms
6:	learn: 0.9762891	total: 5.83ms	remaining: 6.66ms
7:	learn: 0.9608643	total: 6.57ms	remaining: 5.75ms
8:	learn: 0.9458134	total: 7.37ms	remaining: 4.92ms
9:	learn: 0.9306501	total: 8.11ms	remaining: 4.05ms
10:	learn: 0.9163152	total: 8.86ms	remaining: 3.22ms
11:	learn: 0.9018678	total: 9.48ms	remaining: 2.37ms
12:	learn: 0.8881964	total: 10.2ms	remaining: 1.57ms
13:	learn: 0.8744128	total: 10.8ms	remaining: 774us
14:	learn: 0.8613576	total: 11.6ms	remaining: 0us
Accuracy: 1.0
Importance: 
   Feature Id  Importances
0          4    43.618934
1          3    28.198692
2          2    28.181315
3          5     0.000785
4          1     0.000182
5          0     0.000092


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [53]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [54]:
model = CatBoostClassifier(iterations=15, learning_rate=0.01, depth=10, loss_function='MultiClass')
model.fit(X_train, y_age_train)
y_pred = model.predict(X_test).reshape(-1,)
print("Accuracy:", np.mean(y_pred == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

0:	learn: 1.0948951	total: 308ms	remaining: 4.31s
1:	learn: 1.0911074	total: 569ms	remaining: 3.69s
2:	learn: 1.0874122	total: 830ms	remaining: 3.32s
3:	learn: 1.0836067	total: 1.1s	remaining: 3.02s
4:	learn: 1.0798035	total: 1.36s	remaining: 2.73s
5:	learn: 1.0761241	total: 1.62s	remaining: 2.43s
6:	learn: 1.0727250	total: 1.89s	remaining: 2.16s
7:	learn: 1.0693649	total: 2.15s	remaining: 1.89s
8:	learn: 1.0658729	total: 2.42s	remaining: 1.62s
9:	learn: 1.0623679	total: 2.69s	remaining: 1.34s
10:	learn: 1.0590924	total: 2.99s	remaining: 1.09s
11:	learn: 1.0559910	total: 3.26s	remaining: 815ms
12:	learn: 1.0529779	total: 3.52s	remaining: 542ms
13:	learn: 1.0495648	total: 3.79s	remaining: 271ms
14:	learn: 1.0465115	total: 4.06s	remaining: 0us
Accuracy: 0.6305170239596469
Most important features:
1. ovsyanochan
2. mudakoff
3. styd.pozor
4. 4ch
5. rhymes
6. pixel_stickers
7. tumblr_vacuum
8. reflexia_our_feelings
9. pustota_diary
10. fuck_humor


#### Пол

In [55]:
model = CatBoostClassifier(iterations=15, learning_rate=0.01, depth=10, loss_function='MultiClass')
model.fit(X_train, y_sex_train)
y_pred = model.predict(X_test).reshape(-1,)
print("Accuracy:", np.mean(y_pred == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

0:	learn: 0.6902555	total: 219ms	remaining: 3.07s
1:	learn: 0.6876998	total: 419ms	remaining: 2.73s
2:	learn: 0.6851113	total: 467ms	remaining: 1.87s
3:	learn: 0.6824323	total: 669ms	remaining: 1.84s
4:	learn: 0.6797071	total: 866ms	remaining: 1.73s
5:	learn: 0.6770563	total: 1.04s	remaining: 1.56s
6:	learn: 0.6745194	total: 1.23s	remaining: 1.41s
7:	learn: 0.6719190	total: 1.41s	remaining: 1.23s
8:	learn: 0.6693069	total: 1.58s	remaining: 1.06s
9:	learn: 0.6666918	total: 1.76s	remaining: 882ms
10:	learn: 0.6641932	total: 1.94s	remaining: 705ms
11:	learn: 0.6615513	total: 2.12s	remaining: 531ms
12:	learn: 0.6591551	total: 2.3s	remaining: 353ms
13:	learn: 0.6566015	total: 2.47s	remaining: 177ms
14:	learn: 0.6541784	total: 2.65s	remaining: 0us
Accuracy: 0.8108448928121059
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. igm
5. modnailru
6. cook_good
7. thesmolny
8. i_d_t
9. reflexia_our_feelings
10. zerofat
